# 7.5 Computed parameters

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


It is seldom possible to arrange that the data values available to a model are precisely
the coefficient values required by the objective and constraints. Even in the simple production
model of [Figure 1-4](../01/tut_1_4.ipynb#fig-1-4a), for example, we wrote the constraint as
```
sum {p in PROD} (1/rate[p]) * Make[p] <= avail;
```
because production rates were given in tons per hour, while the coefficient of `Make[p]`
had to be in hours per ton. Any parameter expression may be used in the constraints and
objective, but the expressions are best kept simple. When more complex expressions are
needed, the model is usually easier to understand if new, computed parameters are
defined in terms of the data parameters.

The declaration of a computed parameter has an assignment phrase, which resembles
the restriction phrase described in the previous section except for the use of an = operator
to indicate that the parameter is being set equal to a certain expression, rather than merely
being restricted by an inequality. As a first example, suppose that the data values provided
to the multicommodity transportation model of [Figure 4-1](../04/4_1_a_multicommodity_transportation_model.ipynb#fig-4-1)
consist of the total demand for each product, together with each destination's share of demand.
The destinations' shares are percentages between zero and 100, but their sum over all destinations
might not exactly equal 100%, because of rounding and approximation. Thus we declare
data parameters to represent the shares, and a computed parameter equal to their sum:
```
param share {DEST} >= 0, <= 100;
param tot_sh = sum {j in DEST} share[j];
```
We can then declare a data parameter to represent total demands, and a computed parameter
that equals demand at each destination:
```
param tot_dem {PROD} >= 0;
param demand {j in DEST, p in PROD}
       = share[j] * tot_dem[p] / tot_sh;
```
The division by tot_sh acts as a correction factor for a sum not equal to 100%. Once
demand has been defined in this way, the model can use it as in [Figure 4-1](../04/4_1_a_multicommodity_transportation_model.ipynb#fig-4-1):
```
subject to Demand {j in DEST, p in PROD}:
	sum {i in ORIG} Trans[i,j,p] = demand[j,p];
```
We could avoid computed parameters by substituting the formulas for `tot_sh` and
`demand[j,p]` directly into this constraint:
```
subject to Demand {j in DEST, p in PROD}:
  sum {i in ORIG} Trans[i,j,p]
     = share[j] * tot_dem[p] / sum {k in DEST} share[k];
```
This alternative makes the model a little shorter, but the computation of the demand and
the structure of the constraint are both harder to follow.

As another example, consider a scenario for the multiperiod production model ([Figure 4-4](../04/4_3_a_model_of_production_and_transportation.ipynb#fig-4-4))
in which minimum inventories are computed. Specifically, suppose that the inventory
of product `p` for week `t` must be at least a certain fraction of `market[p,t+1]`, the
maximum that can be sold in the following week. We thus use the following declarations
for the data to be supplied:
```
param frac > 0;
param market {PROD,1..T+1} >= 0;
```
and then declare
```
param mininv {p in PROD, t in 0..T} = frac * market[p,t+1];
var Inv {p in PROD, t in 0..T} >= mininv[p,t];
```
to define and use parameters `mininv[p,t]` that represent the minimum inventory of
product `p` for week `t`. AMPL keeps all = definitions of parameters up to date throughout
a session. Thus for example if you change the value of frac the values of all the
mininv parameters automatically change accordingly.

If you define a computed parameter as in the examples above, then you cannot also
specify a data value for it. An attempt to do so will result in an error message:
```
mininv was defined in the model
context: param >>> mininv <<< :=   bands 2 3000
```
However, there is an alternative way in which you can define an initial value for a parameter
but allow it to be changed later.

If you define a parameter using the `default` operator in place of `=`, then the parameter
is initialized rather than defined. Its value is taken from the value of the expression to
the right of the default operator, but does not change if the expression's value later
changes. Initial values can be overridden by data statements, and they also may be
changed by subsequent assignment statements. This feature is most useful for writing
AMPL scripts that update certain values repeatedly, as shown in Section 13.2 xTODO.

If you define a parameter using the operator `default` in place of `=`, then you can
specify values in data statements to override the ones that would otherwise be computed.
For instance, by declaring
```
param mininv {p in PROD, t in 0..T}
   default frac * market[p,t+1];
```
you can allow a few exceptional minimum inventories to be specified as part of the data
for the model, either in a list:
```
mininv = {
    ('bands', 2): 3000,
    ('coils', 2): 2000,
    ('coils', 3): 2000
}
ampl.param['mininv'] = mininv
```
or in a table:
```
market = {
    ('bands', 2): 3000,
    ('coils', 2): 2000,
    ('coils', 3): 2000
}
ampl.param['market'] = market
```

The expression that gives the default value of a parameter is evaluated only when the
parameter's value is first needed, such as when an objective or constraint that uses the
parameter is processed by a solve command.

In most `=` and `default` phrases, the operator is followed by an arithmetic expression
in previously defined sets and parameters (but not variables) and currently defined
dummy indices. Some parameters in an indexed collection may be given a computed or
default value in terms of others in the same collection, however. As an example, you can
smooth out some of the variation in the minimum inventories by defining the `mininv`
parameter to be a running average like this:
```
param mininv {p in PROD, t in 0..T} =
   if t = 0 then inv0[p]
      else 0.5 * (mininv[p,t-1] + frac * market[p,t+1]);
```
The values of `mininv` for week 0 are set explicitly to the initial inventories, while the
values for each subsequent week `t` are defined in terms of the previous week's values.
AMPL permits any "recursive" definition of this kind, but will signal an error if it detects
a circular reference that causes a parameter's value to depend directly or indirectly on
itself.

You can use the phrases defined in this section together with the restriction phrases of
the previous section, to further check the values that are computed. For example the declaration
```
param mininv {p in PROD, t in 0..T}
   = frac * market[p,t+1], >= 0;
```

will cause an error to be signaled if the computed value of any of the mininv parameters
is negative. This check is triggered whenever an AMPL session uses mininv for any
purpose.